In [2]:
import pandas as pd

import torch

from transformers import T5Tokenizer, T5ForConditionalGeneration

from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

import nltk

from nltk.stem import WordNetLemmatizer

from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords

from gensim.models import Word2Vec

from torch.optim import AdamW

In [4]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4dfb911f491ed603a8a789ef26ebd18794290206cb411ba64e61fb3e0f9a5429
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from datasets import Dataset

# List of medical stopwords (e.g., words that are essential in the medical field)
medical_stopwords = set([
    "hemoglobin", "RBC", "WBC", "platelet", "count", "level", "normal", "anemia", "blood", 
    "cells", "oxygen", "immune", "system", "function", "test", "range", "cells/µL", "g/dL", 
    "symptoms", "fatigue", "weakness", "indicates", "may", "suggests", "mild", "serious", "evaluation", "suggest", "levels"
])

# Additional stopwords to retain for context
additional_stopwords = set([
    "is", "are", "the", "in", "at", "of", "to", "a", "an", "and", "or", "on", "for", "with"
])

# Combine medical stopwords with general stopwords
stop_words = stopwords.words('english')
all_stopwords = set(stop_words) | medical_stopwords | additional_stopwords

# Function to preprocess medical reports
def preprocess_medical_text(text):
    # Remove newlines and extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove punctuation, but keep periods, commas, etc.
    tokens = [word for word in tokens if word not in string.punctuation]
    
    # Remove stopwords, but keep medical stopwords and numbers
    tokens = [word for word in tokens if word.lower() not in all_stopwords]
    
    # Reconstruct the text after tokenization and removal
    processed_text = ' '.join(tokens)
    
    return processed_text

# Function to preprocess the dataset
def preprocess_dataset(df):
    # Drop the 'SNO.' column if present
    if 'SNO.' in df.columns:
        df = df.drop(columns=['SNO.'])
    
    # Remove spaces from column names
    df.columns = df.columns.str.replace(' ', '')
    
    # Preprocess the 'SUMMARY' column
    df['SUMMARY'] = df['SUMMARY'].apply(preprocess_medical_text)
    
    return df

# Sample dataset (replace with your actual dataset)
data = pd.read_excel("/kaggle/input/data-set/PROJECT_DATA.xlsx")

# Create DataFrame from the sample data
df = pd.DataFrame(data)

# Preprocess the dataset
processed_df = preprocess_dataset(df)

# Format input text (lab values) and target text (summary) for T5
processed_df['input_text'] = processed_df.apply(
    lambda row: f"MCV: {row['MCV']} MCHC: {row['MCHC']} HB: {row['HB']} RBC: {row['RBC']} WBC: {row['WBC']} "
                f"PLT: {row['PLT']} RDWCV: {row['RDWCV']} NEUTRO: {row['NEUTRO']} LYMPHO: {row['LYMPHO']}", axis=1
)
processed_df['target_text'] = processed_df['SUMMARY']

# Convert the DataFrame to a Dataset format suitable for Hugging Face
dataset = Dataset.from_pandas(processed_df[['input_text', 'target_text']])

# Split into training and test datasets
train_test_split = dataset.train_test_split(test_size=0.2)
train_data = train_test_split['train']
test_data = train_test_split['test']

# Load the tokenizer and model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the data
def tokenize_data(example):
    inputs = tokenizer(example['input_text'], max_length=400, truncation=True, padding="max_length")
    targets = tokenizer(example['target_text'], max_length=400, truncation=True, padding="max_length")
    inputs['labels'] = targets['input_ids']
    return inputs

train_data = train_data.map(tokenize_data, batched=True)
test_data = test_data.map(tokenize_data, batched=True)

# Set format for PyTorch
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Prepare DataLoader for training
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)

# Prepare DataLoader for evaluation
test_dataloader = DataLoader(test_data, batch_size=2)

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Define optimizer (AdamW is commonly used for transformers)
optimizer = AdamW(model.parameters(), lr=3e-5)

# Training loop on GPU
model.train()

for epoch in range(200):
    total_loss = 0

    for batch in tqdm(train_dataloader):
        optimizer.zero_grad()

        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}')

# Save the model after training
model.save_pretrained("./t5_PROJECT_100SCRATCH")
tokenizer.save_pretrained("./t5_PROJECT_100SCRATCH")

print("Training complete and model saved!")


Map:   0%|          | 0/225 [00:00<?, ? examples/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Using device: cuda


100%|██████████| 113/113 [00:14<00:00,  7.63it/s]


...

Epoch 198, Average Loss: 0.1105


100%|██████████| 113/113 [00:15<00:00,  7.09it/s]


Epoch 199, Average Loss: 0.1108


100%|██████████| 113/113 [00:15<00:00,  7.09it/s]


Epoch 200, Average Loss: 0.1116
Training complete and model saved!


In [9]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import torch
from tqdm import tqdm

# Function to evaluate the model
def evaluate_model(model, test_dataloader, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0
    predictions, references = [], []
    
    for batch in tqdm(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

        # Decode the predicted token IDs to text
        pred_ids = outputs.logits.argmax(dim=-1)
        decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Store the predictions and references for BLEU/ROUGE
        predictions.extend(decoded_preds)
        references.extend(decoded_labels)

    # Compute average loss
    avg_loss = total_loss / len(test_dataloader)

    # Calculate BLEU score (using NLTK's corpus_bleu)
    bleu_score = corpus_bleu([[ref.split()] for ref in references], [pred.split() for pred in predictions])

    # Calculate ROUGE score (using rouge-score package)
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, pred in zip(references, predictions):
        scores = scorer.score(ref, pred)
        for key in rouge_scores:
            rouge_scores[key].append(scores[key].fmeasure)
    
    # Calculate average ROUGE scores
    avg_rouge_scores = {key: sum(value) / len(value) for key, value in rouge_scores.items()}

    return avg_loss, bleu_score, avg_rouge_scores

# Evaluate the model on the test dataset
avg_loss, bleu_score, avg_rouge_scores = evaluate_model(model, test_dataloader, device)

# Print the evaluation metrics
print(f"Evaluation Results:")
print(f"Average Loss: {avg_loss:.4f}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"ROUGE Scores: {avg_rouge_scores}")


100%|██████████| 29/29 [00:07<00:00,  4.06it/s]


Evaluation Results:
Average Loss: 0.3406
BLEU Score: 0.2970
ROUGE Scores: {'rouge1': 0.6737388419617204, 'rouge2': 0.4781325274534715, 'rougeL': 0.6462441021264972}


In [10]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./t5_PROJECT_100SCRATCH")
tokenizer = T5Tokenizer.from_pretrained("./t5_PROJECT_100SCRATCH")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Function to generate summary for new input text
def generate_summary(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=400, truncation=True, padding="max_length")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Generate the summary (text) using the trained model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=200)
    
    # Decode the generated tokens to get the summary
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

# Example input text (lab values)
input_text = "MCV: 85 MCHC: 33 HB: 13.5 RBC: 4.5 WBC: 7.0 PLT: 250 RDWCV: 15 NEUTRO: 60 LYMPHO: 30"

# Generate the summary for the input text
summary = generate_summary(input_text)
print("Generated Summary:", summary)


Using device: cuda
Generated Summary: MCV 85 fL 13.5 g/dL Platelets low 250 thousand cells/L could affect clotting ability RDW-CV 15 variability red cell sizes Severe requires urgent medical address potential health risks


In [12]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./t5_PROJECT_100SCRATCH")
tokenizer = T5Tokenizer.from_pretrained("./t5_PROJECT_100SCRATCH")

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Function to generate summary for new input text
def generate_summary(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=900, truncation=True, padding="max_length")
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    # Generate the summary (text) using the trained model
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=200)
    
    # Decode the generated tokens to get the summary
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return summary

# Example input text (lab values)
input_text = "MCV: 85 MCHC: 33 HB: 9.5 RBC: 2.5 WBC: 7.0 PLT: 750 RDWCV: 15 NEUTRO: 60 LYMPHO: 30"

# Generate the summary for the input text
summary = generate_summary(input_text)
print("Generated Summary:", summary)


Using device: cuda
Generated Summary: 9.5 g/dL RBC 2.5 million cells/L indicating good transport WBC 7 thousand cells/L neutrophils 60 lymphocytes 30 750 thousand cells/L adequate clotting RDW-CV 15 variation red cell sizes Overall profile shows good health minor concerns regarding
